In [1]:
from models.MAML import MAML
from kitti_utils import Draw_Boxes
from kitti import DataGenerator
import time
import torch
from datasets import load_metric
import PIL as Image 
import matplotlib.pyplot as plt
from matplotlib import patches
import cv2
from torchvision import transforms

In [2]:
label_path = r'datasets/kitti/all_data/Cyclist/labels/000266.txt'
img_path = r'datasets/kitti/all_data/Cyclist/000266'
# change above files id to visualise other images.s



draw = Draw_Boxes()
draw.show_sequence_rgb(label_path, img_path, vis_2dbox=True, vis_3dbox=False, save_img=True, save_path=None, wait_time=30)
time.sleep(0.5)

draw.show_sequence_rgb(label_path, img_path, vis_2dbox=False, vis_3dbox=True, save_img=True, save_path=None, wait_time=30)

# Press q to exit the cv2 windows. Don't close them using the cross.

QObject::moveToThread: Current thread (0x7d30080) is not the object's thread (0x8218ed0).
Cannot move to target thread (0x7d30080)

QObject::moveToThread: Current thread (0x7d30080) is not the object's thread (0x8218ed0).
Cannot move to target thread (0x7d30080)

QObject::moveToThread: Current thread (0x7d30080) is not the object's thread (0x8218ed0).
Cannot move to target thread (0x7d30080)

QObject::moveToThread: Current thread (0x7d30080) is not the object's thread (0x8218ed0).
Cannot move to target thread (0x7d30080)

QObject::moveToThread: Current thread (0x7d30080) is not the object's thread (0x8218ed0).
Cannot move to target thread (0x7d30080)

QObject::moveToThread: Current thread (0x7d30080) is not the object's thread (0x8218ed0).
Cannot move to target thread (0x7d30080)

QObject::moveToThread: Current thread (0x7d30080) is not the object's thread (0x8218ed0).
Cannot move to target thread (0x7d30080)

QObject::moveToThread: Current thread (0x7d30080) is not the object's thread

(248, 169) (354, 331)
(354, 174) (430, 295)
Save path:  ./logs/kitti/vis/2D_box/000266.png


QObject::moveToThread: Current thread (0x7d30080) is not the object's thread (0x8218ed0).
Cannot move to target thread (0x7d30080)

QObject::moveToThread: Current thread (0x7d30080) is not the object's thread (0x8218ed0).
Cannot move to target thread (0x7d30080)

QObject::moveToThread: Current thread (0x7d30080) is not the object's thread (0x8218ed0).
Cannot move to target thread (0x7d30080)

QObject::moveToThread: Current thread (0x7d30080) is not the object's thread (0x8218ed0).
Cannot move to target thread (0x7d30080)

QObject::moveToThread: Current thread (0x7d30080) is not the object's thread (0x8218ed0).
Cannot move to target thread (0x7d30080)

QObject::moveToThread: Current thread (0x7d30080) is not the object's thread (0x8218ed0).
Cannot move to target thread (0x7d30080)

QObject::moveToThread: Current thread (0x7d30080) is not the object's thread (0x8218ed0).
Cannot move to target thread (0x7d30080)

QObject::moveToThread: Current thread (0x7d30080) is not the object's thread

Save path:  ./logs/kitti/vis/3D_box/000266.png


In [ ]:
image_path = "./datasets/kitti/training/image_2/000098.png"
# image = Image.open(imagqe_path)
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

image_tensor = transforms.ToTensor()(image_rgb)  # Return a 0-1 Normalised image, using the height and width

cv2.imshow('OG Image', image) 


while True:
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cv2.destroyAllWindows()

In [ ]:

path = r"./datasets/kitti/all_data/"
meta_train_iterable = DataGenerator(num_classes=2, num_samples_per_class=3, images_folder_path=path, class_names=None, meta_train=True, cache=True)



In [ ]:
def visualise_detection(model, tensor_image : torch.Tensor, results: dict):

    fig, ax = plt.subplots(1)

    # Display the image
    ax.imshow(tensor_image.permute(1, 2, 0))

    # Note results contains scores, labels and boxes for a single image.

    # Bounding Box: (left_top_x, left_top_y, right_bottom_x, right_bottom_y)
    # Label : Integer Convert to label using model.config.id2label[label.item()]
    # Score : Probability of Box having the given labelled image.    
    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
            
            box = [round(i, 2) for i in box.tolist()]

            # We need the height and width for the bounding box. 
            width = box[2] - box[0]
            height = box[1] - box[3]
            

            print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
            )
            
            # This Rectangle function takes ((left_bottom_x,left_bottom_y), width , height)

            rect = patches.Rectangle((box[0], box[3]), width, height, linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)

    plt.show()


In [ ]:
import glob 
from torchvision import transforms
from transformers import DetrImageProcessor, DetrForObjectDetection
files = glob.glob("./datasets/kitti/training/image_2/*")


processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")


for path in files: 


    image = Image.open(path)
    # tensor_image = transform(image)
    

    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)

    target_sizes = torch.tensor([image.size[::-1]])
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.95)[0]

    visualise_detection(model, tensor_image,results)


In [ ]:
meta_train_loader = iter(
    torch.utils.data.DataLoader(
        meta_train_iterable,
        batch_size=1,
        pin_memory=True,
    )
)